In [ ]:
import numpy as np
import pandas as pd
import tqdm
import re
import io
#from lightfm import LightFM
from io import BytesIO
from PIL import Image
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import gensim
from scipy.sparse import coo_matrix, csr_matrix
from scipy.sparse.linalg import svds
from google.colab import files
uploaded=files.upload()

Saving BX-Book-Ratings.csv to BX-Book-Ratings.csv
Saving BX-Books.csv to BX-Books.csv
Saving BX-Users.csv to BX-Users.csv


In [ ]:
books = pd.read_csv("BX-Books.csv")

interactions = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding = "ISO-8859-1")
interactions = interactions[interactions["Book-Rating"] != 0]
interactions


<ipython-input-2-6c5b329464f2>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv")


,User-ID,ISBN,Book-Rating
1,276726,0155061224,5
3,276729,052165615X,3
4,276729,0521795028,6
6,276736,3257224281,8
7,276737,0600570967,6
...,...,...,...
1149773,276704,0806917695,5
1149775,276704,1563526298,9
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [ ]:

books


,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [ ]:
books_meets = interactions.groupby("ISBN")["User-ID"].count().reset_index().rename(columns={"User-ID": "user_num"})

In [ ]:
user_meets = interactions.groupby("User-ID")["ISBN"].count().reset_index().rename(columns={"ISBN": "books_num"})

In [ ]:
interactions = interactions.merge(books_meets, on=["ISBN"]).merge(user_meets, on=["User-ID"])

In [ ]:
interactions = interactions[(interactions["user_num"] > 5) &
                            (interactions["books_num"] > 5) &
                            (interactions["books_num"] < 200)]

In [ ]:
users = pd.read_csv('BX-Users.csv', delimiter=';', encoding = 'ISO-8859-1')

In [ ]:
interactions = interactions.merge(books[["ISBN", "Image-URL-M", "Book-Title"]].rename(
    columns={"Image-URL-M": "picture_url"}), on=["ISBN"])

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
interactions["product_id"] = le.fit_transform(interactions["ISBN"])
interactions["vid"] = le.fit_transform(interactions["User-ID"])
interactions = interactions.merge(users[["User-ID", "Location", "Age"]].rename(
    columns={"Location": "loc"}), on=["User-ID"])
#interactions['avg']=interactions.ISBN.map(interactions.groupby(['ISBN']).Age.mean())
#interactions.assign(Avg=interactions.groupby('ISBN').transform('mean'))

In [ ]:
#for i in interactions["product_id"]:
#  interactions["avg"] = interactions[interactions["product_id"]==i].mean()
interactions.head()

,User-ID,ISBN,Book-Rating,user_num,books_num,picture_url,Book-Title,product_id,vid,loc,Age
0,86583,3404139178,9,8,18,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,3445,"stuttgart, , germany",31.0
1,86583,3453061187,8,13,18,http://images.amazon.com/images/P/3453061187.0...,Die Jury. Roman.,10495,3445,"stuttgart, , germany",31.0
2,86583,3426609061,7,8,18,http://images.amazon.com/images/P/3426609061.0...,House Of God.,10428,3445,"stuttgart, , germany",31.0
3,132500,3404139178,10,8,43,http://images.amazon.com/images/P/3404139178.0...,Das Lacheln der Fortuna: Historischer Roman,10397,5248,"fürth, ,",29.0
4,132500,3499232871,8,11,43,http://images.amazon.com/images/P/3499232871.0...,Herzsprung,10552,5248,"fürth, ,",29.0


In [ ]:
csr_rates = coo_matrix((interactions["Book-Rating"], (interactions["vid"], interactions["product_id"])),
                            shape=(len(set(interactions["vid"])), len(set(interactions["product_id"]))))
print(csr_rates)

  (3445, 10397)	9
  (3445, 10495)	8
  (3445, 10428)	7
  (5248, 10397)	10
  (5248, 10552)	8
  (5248, 10514)	8
  (5248, 10402)	6
  (5248, 10388)	5
  (2622, 10397)	10
  (2622, 10595)	10
  (2622, 10574)	10
  (2622, 10596)	10
  (2622, 10573)	10
  (2622, 10575)	10
  (2622, 10576)	10
  (2622, 10566)	10
  (2622, 10560)	4
  (10887, 10397)	9
  (4228, 10397)	6
  (4228, 10558)	9
  (4228, 10368)	7
  (6225, 10397)	9
  (6225, 10476)	2
  (6225, 8716)	10
  (6225, 10565)	8
  :	:
  (5438, 10698)	9
  (5352, 10666)	8
  (6503, 795)	8
  (1938, 10698)	8
  (1938, 10699)	7
  (84, 10685)	10
  (84, 10688)	10
  (84, 10689)	10
  (5557, 10685)	5
  (7758, 10685)	8
  (7758, 10689)	10
  (1620, 10688)	8
  (2272, 10689)	6
  (202, 10703)	8
  (943, 10699)	10
  (4213, 10699)	8
  (5786, 10350)	8
  (5669, 10691)	7
  (755, 10705)	9
  (6518, 10702)	7
  (879, 7079)	8
  (7498, 6211)	7
  (8112, 10209)	9
  (8754, 10138)	8
  (2964, 9496)	8


In [ ]:
from sklearn import preprocessing

users=users.drop_duplicates()
users['Age']=users['Age'].fillna(0).astype(int)

users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",0
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",0
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",0


In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357247 sha256=f2e210c4f822398e3a0dc1df0a5a58254e71c0dad2c37b564dc213d13ea7740a
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from io import BytesIO
from PIL import Image
def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def rec_imaging(product_ids, content_dict, measure=None,  top_n=5):
    picture_urls = [content_dict[i] for i in product_ids]
    #book_name=[book_dict[i] for i in product_ids]
    cnt = 0
    for idx, chunk in enumerate(chunks(picture_urls, top_n)):
        fig = plt.figure(figsize=(20, 4))
        for n, i in enumerate(chunk):
            try:
                r = requests.get(i)
                im = Image.open(BytesIO(r.content))

            except:
                print('Something went wrong with {url}'.format(url=i))

                continue

            a = fig.add_subplot(1, top_n, n + 1)
            if measure is not None:
                a.title.set_text("measure = {}".format(np.round(measure[cnt], 4)))
                cnt += 1
            plt.imshow(im)
            plt.axis('off')
        plt.show()

In [ ]:
class Top():
    def __init__(self, interactions):
        # Создаём словарь для того, чтобы по product_id возвращать url товара
        self.product_id_to_url = {}
        for i in interactions[["product_id", "picture_url"]].drop_duplicates().values:
            self.product_id_to_url[i[0]] = i[1]
        self.interactions = interactions

    def top_items(self,age,location):
        # Хотим получить в отсортированном порядке датафрейм - product_id, user_number
        items = self.interactions[self.interactions["Book-Rating"]>=8 ]
        items = items[items["loc"]==location]
        items = items[items["Age"]==age ]\
                                                .groupby("product_id")["vid"].count() \
                                                .reset_index() \
                                                .sort_values("vid", ascending=False)[:30]

        rec_imaging(items["product_id"].values, self.product_id_to_url, items["vid"].values)

**ПУНКТ 1 персональный топ**

In [ ]:
age=25
location="omaha, nebraska, usa"
T = Top(interactions)
T.top_items(age,location)

# Фотографии почему то не отображаются

Something went wrong with http://images.amazon.com/images/P/0345350499.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0679735909.01.MZZZZZZZ.jpg
Something went wrong with http://images.amazon.com/images/P/0786867906.01.MZZZZZZZ.jpg


<Figure size 2000x400 with 0 Axes>

**ПУНКТ 2 кластеризация пользователей**

In [ ]:
import math
import numpy as np
def sim(u,v):
    return np.dot(u,v)/(math.sqrt(np.dot(u,u))*math.sqrt(np.dot(v,v)))
#sim([1,1,-1],[1,1,-1])

Создадим датасет, где строка - id_visitor, а столбец - id_product. Затем с помощью алгоритма К ближайших соседей разобьем на 20 кластеров пользователей, индексы десяти максимальных значений центров кластеров и будут являться топом книг для исходного кластера

In [ ]:
from operator import index
mas_interactions=np.zeros((max(interactions['vid']),max(interactions['product_id'])))
for i in interactions.index:
  #print(interactions['vid'][i])
  mas_interactions[interactions['vid'][i]-1][interactions['product_id'][i]-1]=interactions['Book-Rating'][i]
mas_interactionsDataset=pd.DataFrame(mas_interactions)
mas_interactionsDataset

,0,1,2,3,4,5,6,7,8,9,...,10701,10702,10703,10704,10705,10706,10707,10708,10709,10710
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
from sklearn.cluster import KMeans

kmeans=KMeans(n_clusters=20)
kmeans.fit(mas_interactions)

print(kmeans.cluster_centers_[0])


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


[0. 0. 0. ... 0. 0. 0.]


In [ ]:
def top10books_from20clusters(cluster_centers):
  arr_top_books=np.zeros((20,10))
  k=0
  for i in cluster_centers:
    for j in range(10):
      max_index=np.argmax(i)
      i[max_index]=0
      arr_top_books[k][j]=int(max_index)
    #print(np.max(i))
    k+=1
  return arr_top_books

top10=top10books_from20clusters(kmeans.cluster_centers_)
#print(top10)
def id_book_to_name_book(lst):
  lst_name=np.array(["j"])
  for i in lst:
    for j in interactions.index:
      if (i==interactions['product_id'][j]):
        print(interactions['Book-Title'][j])
        break
print("Top 10 books of the first cluster:  ")
print(id_book_to_name_book(top10[1]))


Top 10 books of the first cluster:  
Meeting Luciano (Ballantine Readers Circle)
The Horse Whisperer
The Runaway Jury
Hotel World
Oh, the Thinks You Can Think! (I Can Read It All by Myself Beginner Books)
The Reef
The Dirty Duck
Black Sunday
Daddy
Lost Souls
None


**ПУНКТ 4 коллоборативная фильтрация**


In [ ]:
user_rate=mas_interactions.iloc[456].values
metric=cosine_similarity([user_rate],mas_interactions.values).reshape(-1,1)

In [ ]:
rates = mas_interactions.values * metric
rates

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
pd.DataFrame(rates)

,0,1,2,3,4,5,6,7,8,9,...,10701,10702,10703,10704,10705,10706,10707,10708,10709,10710
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10954,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10955,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10956,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
total_rate = (1 - user_rate) * np.array(np.sum(rates, axis=0))

In [ ]:
pd.DataFrame([total_rate])

,0,1,2,3,4,5,6,7,8,9,...,10701,10702,10703,10704,10705,10706,10707,10708,10709,10710
0,0.0,0.0,0.0,0.62966,0.0,1.370082,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.048664,0.0


Не успел доделать, нашел лишь рекомендации книг для пользователя с индексом 456